# Description

This notebook will be the same as [notebook #3](https://github.com/nebojsa55/Computational-Genomics_MidTerm-Project/blob/master/notebooks/3.%20Linear-regression.ipynb), except that the model was now tested on the whole testing set (368 samples) to observe whether the model is viable.

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Navigate to the folder cointaing our data
%cd 'drive/MyDrive/ETF/Master/Computational-Genomics/Project/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ETF/Master/Computational-Genomics/Project/data


In [2]:
anno = pd.read_csv('anoSC1_v11_nokey_with_test_samples.csv', delimiter = ',', index_col = 0)
HTA20_RMA = pd.read_csv('HTA20_RMA.csv', delimiter = ',', index_col = 0).transpose()

# Sync the X and y data by sorting the labels

df1 = anno.sort_index()
df2 = HTA20_RMA.sort_index()


In [3]:
# Check to see if the indexes are the same
(df1.index == df2.index).all()

True

In [4]:
# Drop Sample_X samples

X = df2.iloc[32:,:]
y = df1.iloc[32:,:]

In [5]:
# Get training and test sets

X_train = X.loc[y['Train'] == 1,:]
X_test = X.loc[y['Train'] == 0,:]

y_train = y.loc[y['Train'] == 1,:]
y_test = y.loc[y['Train'] == 0,:]

In [6]:
from sklearn.preprocessing import StandardScaler

XX_train = np.zeros(X_train.shape)
for i in [1,2,3,4,5,6,7,8,9,10]:
    scale = StandardScaler()
    indices = np.bool8(y_train['Batch'] == i)
    Xtemp = X_train.iloc[indices,:]
    scale.fit(Xtemp)
    XX_train[indices,:] = scale.transform(Xtemp)

# delete batch column
yy_train = y_train['GA']

In [7]:
XX_test = np.zeros(X_test.shape)
for i in [1,2,3,4,5,6,7,8,9,10]:
    scale = StandardScaler()
    indices = np.bool8(y_test['Batch'] == i)
    Xtemp = X_test.iloc[indices,:]
    scale.fit(Xtemp)
    XX_test[indices,:] = scale.transform(Xtemp)

# delete batch column
yy_test = y_test['GA']

In [8]:
from sklearn.feature_selection import f_regression,SelectKBest
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.model_selection import KFold
import warnings 

# Create the scorer for cross_val_score
scorer = make_scorer(mean_squared_error, greater_is_better = False)

# Create KFold 
kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)

# Create our linear model
lin_model = ElasticNetCV(cv = kfold, random_state = 42)

# Create lists where cv score will be saved
cv_lin = []

# Ignore convergence warnings
warnings.filterwarnings("ignore")

select = SelectKBest(f_regression, k = 1000)
X_best = select.fit_transform(XX_train,yy_train)
  
lin_model.fit(X_best,yy_train)

yy_pred = lin_model.predict(select.transform(XX_test))

from sklearn.metrics import mean_squared_error

loss = mean_squared_error(yy_test,yy_pred,squared = False)


In [9]:
loss

5.040064449436394